<a href="https://colab.research.google.com/github/tobiascz/demotime/blob/main/Gradio%2BGradCAM%2Bmobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pytorch

In [ ]:
# import pytorch related dependencies
import torch
from PIL import Image
from torch import nn
import numpy as np
import torchvision as torchvision
import torchvision.transforms as transforms

In [ ]:
# model setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classes = [ 'actinic keratoses', 'basal cell carcinoma', 'benign keratosis-like lesions', 
           'dermatofibroma','melanoma', 'melanocytic nevi', 'vascular lesions']
model = torchvision.models.mobilenet_v3_large(pretrained = False) # This is a very well known network but it is designed for 1000 classes and not just cats and dogs this is why we need the next line
model.classifier[3] = nn.Linear(1280, 7)
state_dict_trained = torch.hub.load_state_dict_from_url("https://github.com/tobiascz/demotime/raw/main/checkpoints/ham10k_checkpoint_mobile_0.82_epoch24.pt", model_dir=".", map_location = device)
model.load_state_dict(state_dict_trained["model_state_dict"]) ## Here we load the trained weights (state_dict) in our model 
model.eval() # This

In [ ]:
# image pre-processing
norm_mean = (0.4914, 0.4822, 0.4465)
norm_std = (0.2023, 0.1994, 0.2010)
transform = transforms.Compose([ # resize image to the network input size
                  transforms.CenterCrop((400,400)),
                  transforms.ToTensor(),
                  transforms.Normalize(norm_mean, norm_std)
                  ])
# convert tensot to numpy array
def tensor2npimg(tensor, mean, std):
  # inverse of normalization
  tensor = tensor.clone()
  mean_tensor = torch.as_tensor(list(mean), dtype=tensor.dtype, device=tensor.device).view(-1,1,1)
  std_tensor = torch.as_tensor(list(std), dtype=tensor.dtype, device=tensor.device).view(-1,1,1)
  tensor.mul_(std_tensor).add_(mean_tensor)
  # convert tensor to numpy format for plt presentation
  npimg = tensor.numpy()
  npimg = np.transpose(npimg,(1,2,0)) # C*H*W => H*W*C
  return npimg

GradCAM

In [ ]:
# clone repository
!git clone https://github.com/jacobgil/pytorch-grad-cam.git
# install requirements
!pip install -r /content/pytorch-grad-cam/requirements.txt
# move core module to working directiory
!mv /content/pytorch-grad-cam/pytorch_grad_cam /content/pytorch_grad_cam

In [ ]:
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
# draw Grad-CAM on image
# target layer could be any layer before the final attention block
# Some common choices are:
#   FasterRCNN: model.backbone
#   Resnet18 and 50: model.layer4[-1]
#   VGG and densenet161: model.features[-1]
#   mnasnet1_0: model.layers[-1]
#   ViT: model.blocks[-1].norm1
#   SwinT: model.layers[-1].blocks[-1].norm1
def image_grad_cam(model, input_tensor, input_float_np, target_layers):
  cam = GradCAM(model=model, target_layers=target_layers, use_cuda=False)
  grayscale_cam = cam(input_tensor=input_tensor, aug_smooth=True, eigen_smooth=True)
  grayscale_cam = grayscale_cam[0, :]
  return show_cam_on_image(input_float_np, grayscale_cam, use_rgb=True)

Gradio

In [ ]:
# install dependencies
!pip install gradio
import gradio as gr

In [ ]:
# download sample images
!wget https://4ag46i294nta1038p13v77x1-wpengine.netdna-ssl.com/wp-content/uploads/basal-cell-carcinoma-2-open-sore.png
!mv /content/basal-cell-carcinoma-2-open-sore.png /content/sample1.png
!wget https://www.researchgate.net/profile/Kabir-Sardana/publication/261140754/figure/fig11/AS:669586957541379@1536653379254/Raised-pigmented-papular-lesion-pathognomic-of-a-compound-melanocytic-nevi.jpg
!mv /content/Raised-pigmented-papular-lesion-pathognomic-of-a-compound-melanocytic-nevi.jpg /content/sample2.jpg
!wget https://images.medicinenet.com/images/slideshow/skin_cancer_s2_actinic_keratosis.jpg
!mv /content/skin_cancer_s2_actinic_keratosis.jpg /content/sample3.jpg

In [ ]:
# config the predict function for Gradio, input type of image is numpy.nparray
def predict(input_img):
  # numpy.nparray -> PIL.Image
  leasionExample = Image.fromarray(input_img.astype('uint8'), 'RGB')
  # normalize the image to fit the input size of our model
  leasion_tensor = transform(leasionExample)
  input_float_np = tensor2npimg(leasion_tensor, norm_mean, norm_std)
  leasion_tensor = leasion_tensor.unsqueeze(dim=0)
  # predict
  with torch.no_grad():
    outputs = model(leasion_tensor)
  outputs = torch.exp(outputs) 
  # probabilities of all classes
  pred_softmax = torch.softmax(outputs, dim=1).cpu().numpy()[0]
  # class with hightest probability
  pred = torch.argmax(outputs, dim=1).cpu().numpy()
  # diagnostic suggestions
  if pred == 1 or pred == 4:
    suggestion = "CHECK WITH YOUR MD!"
  else:
    suggestion = "Nothing to be worried about."
  # grad_cam image
  target_layers = model.features[-1]
  output_img = image_grad_cam(model,leasion_tensor,input_float_np,target_layers)
  # return label dict and suggestion
  return {classes[i]: float(pred_softmax[i]) for i in range(len(classes))}, suggestion, output_img

In [ ]:
# start gradio application
gr.Interface(
        fn=predict, 
        inputs=gr.inputs.Image(), 
        outputs=[gr.outputs.Label(label="Predict Result"), gr.outputs.Textbox(type="str", label="Recommendation"), gr.outputs.Image(label="GradCAM")],
        examples=[['sample1.png'],['sample2.jpg'],['sample3.jpg']],
        title="Skin Lesion Classifier"
      ).launch()